# Limpieza de datos

In [3]:
#librerias
import pandas as pd

### Cargar data

In [2]:
df_historical_data = pd.read_csv('fifa_worldcup_historical_data.csv')#all historical
df_fixture = pd.read_csv("fifa_worldcup_fixture.csv")#matches from 2022
df_missing = pd.read_csv("fifa_worldcup_missing_data.csv")#missing data

In [3]:
df_historical_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   home    865 non-null    object
 1   score   865 non-null    object
 2   away    865 non-null    object
 3   year    865 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 27.2+ KB


In [27]:
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

### Clean df_missing and add to historical data

In [28]:
#df_missing[df_missing['home'].isnull()]
df_missing.dropna(inplace=True)
df_missing.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, 0 to 35
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   home    36 non-null     object
 1   score   36 non-null     object
 2   away    36 non-null     object
 3   year    36 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 1.4+ KB


In [29]:
#add to historical data
df_all_historical =  pd.concat([df_historical_data,df_missing], ignore_index = True)
df_all_historical

,home,score,away,year
0,France,4–1,Mexico,1930
1,Argentina,1–0,France,1930
2,Chile,3–0,Mexico,1930
3,Chile,1–0,France,1930
4,Argentina,6–3,Mexico,1930
...,...,...,...,...
896,Netherlands,1–1,Egypt,1990
897,England,0–0,Netherlands,1990
898,Republic of Ireland,0–0,Egypt,1990
899,England,1–0,Egypt,1990


In [30]:
#delete duplicates
df_all_historical.drop_duplicates(inplace = True)
df_all_historical.sort_values('year', inplace = True)
df_all_historical

,home,score,away,year
0,France,4–1,Mexico,1930
17,Uruguay,4–2,Argentina,1930
16,Uruguay,6–1,Yugoslavia,1930
15,Argentina,6–1,United States,1930
14,Paraguay,1–0,Belgium,1930
...,...,...,...,...
829,Serbia,0–2,Brazil,2018
828,Serbia,1–2,Switzerland,2018
827,Brazil,2–0,Costa Rica,2018
825,Costa Rica,0–1,Serbia,2018


### Clean all historical

In [31]:
delete_index = df_all_historical[(df_all_historical['home'].str.contains('Sweden')) 
                   & df_all_historical['away'].str.contains('Austria') ].index
#eliminamos este registro ya que no nos aporta info

In [32]:
df_all_historical.drop(index=delete_index, inplace=True)

In [47]:
#df_all_historical[df_all_historical['score'].str.contains('[^\d–]')]
df_all_historical['score'] = df_all_historical['score'].str.replace('[^\d–]','',regex = True)
#eliminamos carcateres especiales del marcador

In [50]:
df_all_historical['home'] = df_all_historical['home'].str.strip()
df_all_historical['away'] = df_all_historical['away'].str.strip()
#quitamos posibles espacios que se encuentren

In [56]:
df_all_historical[['HomeGoals', 'AwayGoals']] = df_all_historical['score'].str.split('–', expand = True)
#utilizamos expand para separar el resultado en dos columnas
df_all_historical

,home,score,away,year,HomeGoals,AwayGoals
0,France,4–1,Mexico,1930,4,1
17,Uruguay,4–2,Argentina,1930,4,2
16,Uruguay,6–1,Yugoslavia,1930,6,1
15,Argentina,6–1,United States,1930,6,1
14,Paraguay,1–0,Belgium,1930,1,0
...,...,...,...,...,...,...
829,Serbia,0–2,Brazil,2018,0,2
828,Serbia,1–2,Switzerland,2018,1,2
827,Brazil,2–0,Costa Rica,2018,2,0
825,Costa Rica,0–1,Serbia,2018,0,1


In [58]:
df_all_historical.drop('score', axis=1, inplace = True)
df_all_historical

,home,away,year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1
17,Uruguay,Argentina,1930,4,2
16,Uruguay,Yugoslavia,1930,6,1
15,Argentina,United States,1930,6,1
14,Paraguay,Belgium,1930,1,0
...,...,...,...,...,...
829,Serbia,Brazil,2018,0,2
828,Serbia,Switzerland,2018,1,2
827,Brazil,Costa Rica,2018,2,0
825,Costa Rica,Serbia,2018,0,1


In [59]:
#renameColumns
df_all_historical.rename(columns = {'home':'HomeTeam','away': 'AwayTeam', 'year': 'Year'}, inplace = True)
df_all_historical.head(1)

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1


In [61]:
df_all_historical = df_all_historical.astype({'HomeGoals': int, 'AwayGoals': int})
#convertimos datos a int

In [64]:
df_all_historical['TotalGoals'] = df_all_historical['HomeGoals'] + df_all_historical['AwayGoals']
df_all_historical

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
17,Uruguay,Argentina,1930,4,2,6
16,Uruguay,Yugoslavia,1930,6,1,7
15,Argentina,United States,1930,6,1,7
14,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
829,Serbia,Brazil,2018,0,2,2
828,Serbia,Switzerland,2018,1,2,3
827,Brazil,Costa Rica,2018,2,0,2
825,Costa Rica,Serbia,2018,0,1,1


### Export clean data

In [65]:
df_all_historical.to_csv('clean_fifa_worldcup_matches.csv', index = False)
df_fixture.to_csv('clean_fifa_worldcup_fixture.csv', index = False)

### Extra verifications

In [4]:
df_all_historical = pd.read_csv('clean_fifa_worldcup_matches.csv')
# verify number of matches per competition
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]

for year in years:
    print(year, len(df_all_historical[df_all_historical['Year']==year]))

1930 18
1934 17
1938 18
1950 22
1954 26
1958 35
1962 32
1966 32
1970 32
1974 38
1978 38
1982 52
1986 52
1990 52
1994 52
1998 64
2002 64
2006 64
2010 64
2014 64
2018 64
